## Salaries for Professors

[R documentation](http://finzi.psych.upenn.edu/R/library/car/html/Salaries.html)

In [ ]:
using Gadfly
using DataFrames
using KernelDensity


## Download the academic salaries data

In [ ]:
;wget https://raw.githubusercontent.com/aviks/ODSC/master/salaries.csv

In [ ]:
SalariesFrame = readtable("salaries.csv")

## Getting basic information about a DataFrame

In [ ]:
size(SalariesFrame)

In [ ]:
names(SalariesFrame)

In [ ]:
head(SalariesFrame, 10)

In [ ]:
tail(SalariesFrame)

In [ ]:
describe(SalariesFrame)

## Basic manipulation of a DataFrame

In [ ]:
sort!(SalariesFrame, cols=[:YrsService, :YrsSincePhD])

In [ ]:
pool!(SalariesFrame, [:Discipline, :Sex])

In [ ]:
levels(SalariesFrame[:Sex])

## Exploratory statistical analysis of professorial salaries

In [ ]:
by(SalariesFrame, :Rank) do df
  DataFrame(
    n = length(df[:Salary]),
    mean = round(mean(df[:Salary]), 3),
    std = round(std(df[:Salary]), 3)
  )
end

In [ ]:
by(SalariesFrame, :Discipline) do df
  DataFrame(
    n = length(df[:Salary]),
    mean = round(mean(df[:Salary]), 3),
    std = round(std(df[:Salary]), 3)
  )
end

In [ ]:
by(SalariesFrame, :Sex) do df
  DataFrame(
    n = length(df[:Salary]),
    mean = round(mean(df[:Salary]), 3),
    std = round(std(df[:Salary]), 3)
  )
end

In [ ]:
by(SalariesFrame, [:Rank, :Discipline]) do df
  DataFrame(
    n = length(df[:Salary]),
    mean = round(mean(df[:Salary]), 3),
    std = round(std(df[:Salary]), 3)
  )
end

In [ ]:
by(SalariesFrame, [:Rank, :Sex]) do df
  DataFrame(
    n = length(df[:Salary]),
    mean = round(mean(df[:Salary]), 3),
    std = round(std(df[:Salary]), 3)
  )
end

In [ ]:
by(SalariesFrame, [:Discipline, :Sex]) do df
  DataFrame(
    n = length(df[:Salary]),
    mean = round(mean(df[:Salary]), 3),
    std = round(std(df[:Salary]), 3)
  )
end

In [ ]:
by(SalariesFrame, [:Rank, :Discipline, :Sex]) do df
  DataFrame(
    n = length(df[:Salary]),
    mean = round(mean(df[:Salary]), 3),
    std = round(std(df[:Salary]), 3)
  )
end

In [ ]:
plot(SalariesFrame, x="Rank", y="Salary", Geom.boxplot)

In [ ]:
plot(SalariesFrame, x="Discipline", y="Salary", Geom.boxplot)

In [ ]:
plot(SalariesFrame, x="Sex", y="Salary", Geom.boxplot)

In [ ]:
hstack(
  plot(SalariesFrame, x="Rank", y="Salary", Geom.boxplot),
  plot(SalariesFrame, x="Discipline", y="Salary", Geom.boxplot),
  plot(SalariesFrame, x="Sex", y="Salary", Geom.boxplot)
)

In [ ]:
plot(SalariesFrame, x="YrsSincePhD", y="Salary")

In [ ]:
plot(SalariesFrame, x="YrsService", y="Salary")

In [ ]:
plot(SalariesFrame, x="YrsSincePhD", y="Salary", color="Rank")

In [ ]:
plot(SalariesFrame, x="YrsService", y="Salary", color="Rank")

In [ ]:
plot(SalariesFrame, x="YrsSincePhD", y="Salary", color="Discipline")

In [ ]:
plot(SalariesFrame, x="YrsService", y="Salary", color="Discipline")

In [ ]:
plot(SalariesFrame, x="YrsSincePhD", y="Salary", color="Sex")

In [ ]:
plot(SalariesFrame, x="YrsService", y="Salary", color="Sex")

In [ ]:
plot(SalariesFrame, x="Salary", Geom.histogram(bincount=50, density=true))

In [ ]:
salaries = kde(SalariesFrame[:Salary])

plot(
  x=salaries.x,
  y=salaries.density,
  Geom.line,
  Guide.xlabel("Salary"),
  Guide.ylabel(""),
  Guide.title("Kernel density estimator")
)

In [ ]:
plot(SalariesFrame, x="Salary", color="Discipline", Geom.histogram(bincount=50, density=true))

In [ ]:
plot(SalariesFrame, x="Salary", color="Rank", Geom.histogram(bincount=50, density=true))

In [ ]:
kdes = [kde(@where(SalariesFrame, :Rank .== l)[:Salary]) for l in levels(SalariesFrame[:Rank])]

In [ ]:
plot(
layer(x=kdes[1].x, y=kdes[1].density, Geom.line, Theme(default_color=colorant"blue")),
layer(x=kdes[2].x, y=kdes[2].density, Geom.line, Theme(default_color=colorant"red")),
layer(x=kdes[3].x, y=kdes[3].density, Geom.line, Theme(default_color=colorant"green")),
Guide.manual_color_key("Rank", ["AsstProf", "AssocProf", "Prof"], ["blue", "red", "green"]),
Guide.xlabel("Salary"), Guide.ylabel(""), Guide.title("Kernel density estimators")
)

In [ ]:
plot(SalariesFrame, x="YrsService", y="Salary", Geom.histogram2d(xbincount=30, ybincount=30))

# Generalised Linear Models

https://github.com/JuliaStats/GLM.jl

In [ ]:
using GLM

In [ ]:
fit(LinearModel, @formula(Salary ~ YrsSincePhD + YrsService), SalariesFrame )

In [ ]:
fit(LinearModel,@formula(Salary ~ YrsSincePhD + YrsService + Sex + Discipline),SalariesFrame )

# Random Forests

https://github.com/bensadeghi/DecisionTree.jl

In [ ]:
using DecisionTree

In [ ]:
model = build_forest(Array(SalariesFrame[:Salary]),
                     Array(SalariesFrame[[:YrsSincePhD,:YrsService]]),
                     2, 10)

In [ ]:
apply_forest(model, Array(SalariesFrame[[:YrsSincePhD,:YrsService]])) .-  SalariesFrame[:Salary]

# Query using Query.jl

In [ ]:
using Query

In [ ]:
q = @from i in SalariesFrame begin
    @where i.Discipline == "A"
    @select {i.Rank,i.Salary}
    @collect DataFrame
end

In [ ]:
q = @from i in SalariesFrame begin
    @group i.Salary by i.Rank into g
    @select {Key=g.key,Avg=sum(g)/length(g)}
    @collect DataFrame
end